In [61]:
# Importing Libraries

In [87]:
import pandas as pd
import numpy as np

In [88]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [89]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [90]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [91]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [92]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [93]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [94]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [95]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [96]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [97]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [98]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [99]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [100]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


In [101]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

In [77]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 32)                5376      
_________________________________________________________________
dropout_6 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [78]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [79]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 25s 3ms/step - loss: 1.2941 - accuracy: 0.4523 - val_loss: 1.1074 - val_accuracy: 0.4924
Epoch 2/30
7352/7352 [==============================] - 23s 3ms/step - loss: 0.9406 - accuracy: 0.5964 - val_loss: 0.8839 - val_accuracy: 0.5938
Epoch 3/30
7352/7352 [==============================] - 25s 3ms/step - loss: 0.7833 - accuracy: 0.6432 - val_loss: 0.7483 - val_accuracy: 0.6125
Epoch 4/30
7352/7352 [==============================] - 28s 4ms/step - loss: 0.6809 - accuracy: 0.6619 - val_loss: 0.7228 - val_accuracy: 0.6169
Epoch 5/30
7352/7352 [==============================] - 27s 4ms/step - loss: 0.6409 - accuracy: 0.6759 - val_loss: 0.7215 - val_accuracy: 0.6328
Epoch 6/30
7352/7352 [==============================] - 26s 3ms/step - loss: 0.5755 - accuracy: 0.7025 - val_loss: 0.7347 - val_accuracy: 0.6586
Epoch 7/30
7352/7352 [==============================] - 31s 4ms/step - loss: 0.558

In [80]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0         0        0                   0   
SITTING                  0      380       101        0                   0   
STANDING                 0       75       457        0                   0   
WALKING                  0        0         1      441                  35   
WALKING_DOWNSTAIRS       0        0         0        0                 415   
WALKING_UPSTAIRS         0        1         0        2                  33   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            27  
SITTING                           10  
STANDING                           0  
WALKING                           19  
WALKING_DOWNSTAIRS                 5  
WALKING_UPSTAIRS                 435  


In [81]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 1s 445us/step


In [82]:
score

[0.5882155524368978, 0.8951476216316223]

- With a simple 2 layer architecture we got 90.09% accuracy and a loss of 0.30
- We can further imporve the performace with Hyperparameter tuning

### Task 1 Use different No of LSTM units

In [83]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 64

In [113]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_14 (LSTM)               (None, 128)               70656     
_________________________________________________________________
dropout_9 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 6)                 774       
Total params: 71,430
Trainable params: 71,430
Non-trainable params: 0
_________________________________________________________________


In [114]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


In [115]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 65s 9ms/step - loss: 1.3818 - accuracy: 0.3885 - val_loss: 1.4502 - val_accuracy: 0.3610
Epoch 2/30
7352/7352 [==============================] - 62s 8ms/step - loss: 1.1629 - accuracy: 0.4943 - val_loss: 1.4035 - val_accuracy: 0.3648
Epoch 3/30
7352/7352 [==============================] - 61s 8ms/step - loss: 1.0246 - accuracy: 0.5706 - val_loss: 1.1373 - val_accuracy: 0.4455
Epoch 4/30
7352/7352 [==============================] - 57s 8ms/step - loss: 0.8515 - accuracy: 0.6096 - val_loss: 1.1883 - val_accuracy: 0.4690
Epoch 5/30
7352/7352 [==============================] - 59s 8ms/step - loss: 0.8079 - accuracy: 0.6160 - val_loss: 0.9130 - val_accuracy: 0.5572
Epoch 6/30
7352/7352 [==============================] - 58s 8ms/step - loss: 0.7202 - accuracy: 0.6582 - val_loss: 0.7115 - val_accuracy: 0.6702
Epoch 7/30
7352/7352 [==============================] - 58s 8ms/step - loss: 0.639

In [116]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 522        0         0        0                   0   
SITTING                  0      403        88        0                   0   
STANDING                 0       92       438        2                   0   
WALKING                  0        0         0      470                  24   
WALKING_DOWNSTAIRS       0        0         0        0                 417   
WALKING_UPSTAIRS         0        0         1        6                   1   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            15  
SITTING                            0  
STANDING                           0  
WALKING                            2  
WALKING_DOWNSTAIRS                 3  
WALKING_UPSTAIRS                 463  


In [117]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 7s 2ms/step


In [118]:
score

[0.2619004359412457, 0.9205971956253052]

### Task 2 Use different No of LSTM units and dropout rates

In [119]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 128
dropout = 0.3

In [120]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(dropout))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, 128)               70656     
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 6)                 774       
Total params: 71,430
Trainable params: 71,430
Non-trainable params: 0
_________________________________________________________________


In [121]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [122]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 105s 14ms/step - loss: 1.2461 - accuracy: 0.4517 - val_loss: 1.3328 - val_accuracy: 0.3899
Epoch 2/30
7352/7352 [==============================] - 104s 14ms/step - loss: 0.8492 - accuracy: 0.6179 - val_loss: 0.7678 - val_accuracy: 0.6780
Epoch 3/30
7352/7352 [==============================] - 648s 88ms/step - loss: 0.6293 - accuracy: 0.7285 - val_loss: 0.5633 - val_accuracy: 0.7883
Epoch 4/30
7352/7352 [==============================] - 79s 11ms/step - loss: 0.3868 - accuracy: 0.8717 - val_loss: 0.3816 - val_accuracy: 0.8395
Epoch 5/30
7352/7352 [==============================] - 67s 9ms/step - loss: 0.2463 - accuracy: 0.9143 - val_loss: 0.3857 - val_accuracy: 0.8527
Epoch 6/30
7352/7352 [==============================] - 65s 9ms/step - loss: 0.2153 - accuracy: 0.9215 - val_loss: 0.2930 - val_accuracy: 0.8836
Epoch 7/30
7352/7352 [==============================] - 68s 9ms/step - loss

In [123]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 515        0         0        0                   0   
SITTING                  3      412        74        0                   0   
STANDING                 0      110       421        0                   0   
WALKING                  0        1         0      469                  14   
WALKING_DOWNSTAIRS       0        1         3        2                 412   
WALKING_UPSTAIRS         0        6         0        8                   2   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            22  
SITTING                            2  
STANDING                           1  
WALKING                           12  
WALKING_DOWNSTAIRS                 2  
WALKING_UPSTAIRS                 455  


In [124]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 5s 2ms/step


In [125]:
score

[0.3174159591353597, 0.9107567071914673]

### Task 3 Use Stacked LSTM units and dropout rates

In [102]:
## increase dropout rate and use stacked LSTM units
# Initializing parameters
epochs = 30
batch_size = 128
n_hidden = 128
dropout = 0.7

In [109]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [110]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 93s 13ms/step - loss: 0.1897 - accuracy: 0.9212 - val_loss: 0.3190 - val_accuracy: 0.9013
Epoch 2/30
7352/7352 [==============================] - 94s 13ms/step - loss: 0.1530 - accuracy: 0.9416 - val_loss: 0.2599 - val_accuracy: 0.9182
Epoch 3/30
7352/7352 [==============================] - 97s 13ms/step - loss: 0.1465 - accuracy: 0.9430 - val_loss: 0.2814 - val_accuracy: 0.9101
Epoch 4/30
7352/7352 [==============================] - 94s 13ms/step - loss: 0.1535 - accuracy: 0.9441 - val_loss: 0.3064 - val_accuracy: 0.9189
Epoch 5/30
7352/7352 [==============================] - 94s 13ms/step - loss: 0.1429 - accuracy: 0.9471 - val_loss: 0.2867 - val_accuracy: 0.9152
Epoch 6/30
7352/7352 [==============================] - 94s 13ms/step - loss: 0.1412 - accuracy: 0.9483 - val_loss: 0.2744 - val_accuracy: 0.9111
Epoch 7/30
7352/7352 [==============================] - 94s 13ms/step - loss

In [111]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))
score = model.evaluate(X_test, Y_test)
score

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  2      408        76        0                   3   
STANDING                 0      109       423        0                   0   
WALKING                  0        0         0      469                  21   
WALKING_DOWNSTAIRS       0        0         0        1                 418   
WALKING_UPSTAIRS         0        0         0        0                   1   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             0  
SITTING                            2  
STANDING                           0  
WALKING                            6  
WALKING_DOWNSTAIRS                 1  
WALKING_UPSTAIRS                 470  
2947/2947 [==============================] - 10s 3ms/step


[0.32517735768649997, 0.9246691465377808]

### Observations

In [126]:
from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["Model","Accuracy", "Loss"]
x.add_row(["Simple LSTM",0.89,0.58 ])
x.add_row(["Modified No of LSTM units", 0.92, 0.26])
x.add_row(["With M_LSTM+ dropout",0.91, 0.31])
x.add_row(["Stacked LSTM", 0.924, 0.32])
print(x)

+---------------------------+----------+------+
|           Model           | Accuracy | Loss |
+---------------------------+----------+------+
|        Simple LSTM        |   0.89   | 0.58 |
| Modified No of LSTM units |   0.92   | 0.26 |
|    With M_LSTM+ dropout   |   0.91   | 0.31 |
|        Stacked LSTM       |  0.924   | 0.32 |
+---------------------------+----------+------+


### Conclusions

#### Stacked LSTM is performed better than other models, Could differentiate activities well when compared to other models